In [1]:
from transformers import TFBertForSequenceClassification, BertTokenizer
import tensorflow as tf
import pandas as pd
from data_preparation import Example, convert_examples_to_tf_dataset, make_batches

## Model setup

In [2]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
#model.bert.trainable = False
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-7)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy()
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('../checkpoints/multibert_sentiment_checkpoint.hdf5', 
                             verbose=1, monitor='val_sparse_categorical_accuracy',
                             save_best_only=True, mode='max', save_weights_only=True)

## Data preparation

In [5]:
train = pd.read_csv("../data/sentiment/en/train.csv", header=None)
train.columns = ["sentiment", "review"]
dev = pd.read_csv("../data/sentiment/en/dev.csv", header=None)
dev.columns = ["sentiment", "review"]
test = pd.read_csv("../data/sentiment/en/test.csv", header=None)
test.columns = ["sentiment", "review"]

In [6]:
train.head()

,sentiment,review
0,1,The Rock is destined to be the st Century 's n...
1,1,The gorgeously elaborate continuation of `` Th...
2,1,Singer\/composer Bryan Adams contributes a sle...
3,1,Yet the act is still charming here .
4,1,Whether or not you 're enlightened by any of D...


Longest sequence in training data

In [7]:
train["review"].apply(lambda x: len(tokenizer(x)["input_ids"])).max()

88

Longest sequence the model allows

In [8]:
# max_length = model.config.max_position_embeddings
# max_length
max_length = 512

We choose the longest input length because test data in other languages can be longer than this

In [9]:
train_dataset = convert_examples_to_tf_dataset([(Example(text=text, category_index=label)) for label, text in train.values], 
                                               tokenizer, max_length=max_length)
dev_dataset = convert_examples_to_tf_dataset([(Example(text=text, category_index=label)) for label, text in dev.values], 
                                             tokenizer, max_length=max_length)
test_dataset = convert_examples_to_tf_dataset([(Example(text=text, category_index=label)) for label, text in test.values], 
                                              tokenizer, max_length=max_length)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


## Training setup

In [10]:
batch_size = 4
repetitions = 20

train_dataset, train_batches = make_batches(train_dataset, batch_size, repetitions)
dev_dataset, dev_batches = make_batches(dev_dataset, batch_size, repetitions)
test_dataset, test_batches = make_batches(test_dataset, batch_size, repetitions=1)

for name, n_batches in zip(["train", "dev", "test"], [train_batches, dev_batches, test_batches]):
    print("{} batches: ".format(name), n_batches)

train batches:  1730
dev batches:  218
test batches:  456


This is what one batch looks like

In [11]:
list(iter(train_dataset))[0]

({'input_ids': <tf.Tensor: shape=(4, 512), dtype=int32, numpy=
  array([[  101, 63719,   100, ...,     0,     0,     0],
         [  101, 10117, 25059, ...,     0,     0,     0],
         [  101, 10313, 10452, ...,     0,     0,     0],
         [  101, 14535, 13028, ...,     0,     0,     0]])>,
  'attention_mask': <tf.Tensor: shape=(4, 512), dtype=int32, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]])>,
  'token_type_ids': <tf.Tensor: shape=(4, 512), dtype=int32, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])>},
 <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 1, 0, 0], dtype=int64)>)

## Training

In [12]:
model.fit(train_dataset, epochs=repetitions, steps_per_epoch=train_batches, 
          validation_data=dev_dataset, validation_steps=dev_batches, 
          callbacks=[checkpoint])

Epoch 1/20
1730/1730 [==============================] - ETA: 0s - loss: 0.6768 - sparse_categorical_accuracy: 0.5708
Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.69954, saving model to ../checkpoints/multibert_sentiment_checkpoint.hdf5
1730/1730 [==============================] - 1148s 664ms/step - loss: 0.6768 - sparse_categorical_accuracy: 0.5708 - val_loss: 0.6135 - val_sparse_categorical_accuracy: 0.6995
Epoch 2/20
1730/1730 [==============================] - ETA: 0s - loss: 0.5853 - sparse_categorical_accuracy: 0.7100
Epoch 00002: val_sparse_categorical_accuracy improved from 0.69954 to 0.77064, saving model to ../checkpoints/multibert_sentiment_checkpoint.hdf5
1730/1730 [==============================] - 1151s 665ms/step - loss: 0.5853 - sparse_categorical_accuracy: 0.7100 - val_loss: 0.5051 - val_sparse_categorical_accuracy: 0.7706
Epoch 3/20
1730/1730 [==============================] - ETA: 0s - loss: 0.5107 - sparse_categorical_accuracy: 0.7632
Epoch 00

## Evaluation

In [ ]:
model.evaluate(test_dataset, steps=test_batches)